In [1]:
import requests
import csv
from datetime import datetime

#### **Load weather and air pollution data for 1 year of each country**

Chosen Cities

In [ ]:
cities = [
    "Seoul", "Busan", "Shanghai", "Tokyo", 
    "New Delhi", "London", "Paris", 
    "New York", "California"
]

기준이 되는 도시인 Busan에 대하여 3년간의 오염 데이터를 수집하고, 이를 통해 baseline 모델을 완성한다.<br>
완성된 모델을 통하여 세계의 도시에 대하여 fine tuning을 진행하여 모델의 일반화 성능을 올린다.

In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta

API_KEY = "64edd4b9daa8d34e8f34965fed3faf75"
base_url = "https://api.openweathermap.org/data/2.5/air_pollution/history"

# 데이터 저장 리스트
all_data = []

for city in cities:
    # 도시의 위도와 경도를 가져오기
    geo_url = f"http://api.openweathermap.org/geo/1.0/direct?q={city}&limit=1&appid={API_KEY}"
    geo_response = requests.get(geo_url).json()
    
    if not geo_response:
        print(f"위치 데이터를 찾을 수 없습니다: {city}")
        continue

    lat = geo_response[0]["lat"]
    lon = geo_response[0]["lon"]

    # 1년 기간 동안 데이터 수집 (7일 단위로)
    start_date = datetime(2021, 1, 1)
    end_date = datetime(2024, 1, 1)

    all_data = []

    while start_date < end_date:
        unix_start = int(start_date.timestamp())
        unix_end = int((start_date + timedelta(days=7)).timestamp())

        params = {
            "lat": lat,
            "lon": lon,
            "start": unix_start,
            "end": unix_end,
            "appid": API_KEY
        }

        response = requests.get(base_url, params=params).json()

        if "list" in response:
            for record in response["list"]:
                all_data.append({
                    "datetime": datetime.fromtimestamp(record["dt"]),
                    "city": city,
                    "lat": lat,
                    "lon": lon,
                    "aqi": record["main"]["aqi"],
                    **record["components"]
                })

        start_date += timedelta(days=7)

    # DataFrame으로 저장
    if all_data:
        df = pd.DataFrame(all_data)
        df.to_csv(f"{city.replace(' ', '_')}_pollution_data.csv", index=False)
        print(f"{city} 데이터 저장 완료!")

Busan 데이터 저장 완료!


In [21]:
# success to get weather data
if response.status_code == 200:
    data = response.json()
    lat = data['city']['coord']['lat']
    lon = data['city']['coord']['lon']

    start_time = int(datetime(2023, 1, 1).timestamp())
    end_time = int(datetime(2024, 1, 1).timestamp())
    
    HISTORICAL_POLLUTION_API_URL = "https://api.openweathermap.org/data/2.5/air_pollution/history"

    
    response_air = requests.get(HISTORICAL_POLLUTION_API_URL, params={
    "lat": lat,
    "lon": lon,
    "start": start_time,
    "end": end_time,
    "appid": API_KEY
    })

    if response_air.status_code == 200:
        print('success')

success


In [24]:
data = response_air.json()

In [29]:
for day in data['list']:
    print(day)

{'main': {'aqi': 2}, 'components': {'co': 198.6, 'no': 0.02, 'no2': 8.91, 'o3': 67.23, 'so2': 3.93, 'pm2_5': 3.17, 'pm10': 3.39, 'nh3': 0.09}, 'dt': 1672498800}
{'main': {'aqi': 2}, 'components': {'co': 200.27, 'no': 0, 'no2': 8.23, 'o3': 67.23, 'so2': 3.67, 'pm2_5': 4.43, 'pm10': 4.68, 'nh3': 0.09}, 'dt': 1672502400}
{'main': {'aqi': 2}, 'components': {'co': 198.6, 'no': 0, 'no2': 8.23, 'o3': 65.8, 'so2': 3.52, 'pm2_5': 6.2, 'pm10': 6.55, 'nh3': 0.09}, 'dt': 1672506000}
{'main': {'aqi': 2}, 'components': {'co': 198.6, 'no': 0, 'no2': 8.23, 'o3': 65.8, 'so2': 3.37, 'pm2_5': 7.58, 'pm10': 8.05, 'nh3': 0.09}, 'dt': 1672509600}
{'main': {'aqi': 2}, 'components': {'co': 198.6, 'no': 0, 'no2': 7.97, 'o3': 65.8, 'so2': 3.34, 'pm2_5': 6.97, 'pm10': 7.41, 'nh3': 0.09}, 'dt': 1672513200}
{'main': {'aqi': 2}, 'components': {'co': 198.6, 'no': 0, 'no2': 7.71, 'o3': 65.8, 'so2': 3.31, 'pm2_5': 5.79, 'pm10': 6.12, 'nh3': 0.1}, 'dt': 1672516800}
{'main': {'aqi': 2}, 'components': {'co': 200.27, 'no'

In [ ]:
# success to get air pollution data
if response_air.status_code == 200:
    data = response_air.json()
    pollution_list = data['list']  # 공기 오염 데이터 리스트

    # CSV 파일 저장
    with open("pollution_data.csv", mode="w", newline="") as file:
        writer = csv.writer(file)
        # CSV 헤더 작성
        writer.writerow(["datetime", "aqi", "co", "no", "no2", "o3", "so2", "pm2_5", "pm10"])

        # 데이터 작성
        for pollution in pollution_list:
            dt = datetime.fromtimestamp(pollution["dt"]).strftime("%Y-%m-%d %H:%M:%S")
            aqi = pollution["main"]["aqi"]
            components = pollution["components"]
            writer.writerow([
                dt, aqi, components["co"], components["no"], components["no2"],
                components["o3"], components["so2"], components["pm2_5"], components["pm10"]
            ])

    print("Pollution data saved to 'pollution_data.csv'.")

Pollution data saved to 'pollution_data.csv'.
